In [1]:

train_msg = open("../../MCMD/java/sort_random_train80_valid10_test10/train.msg.txt", encoding='UTF-8').read().split("\n")

In [2]:
len(train_msg)

360001

In [4]:
def read_first_n_lines(file_path, n):
    with open(file_path, 'r', encoding='UTF-8') as file:
        lines = []
        for i in range(n):
            line = file.readline()
            if not line:
                break
            lines.append(line)
    return lines

# Usage
diff_file_path = '../../MCMD/java/sort_random_train80_valid10_test10/train.diff.txt'
first_1000_lines = read_first_n_lines(diff_file_path, 1000)

In [5]:
first_1000_lines[0]

'mmm a / CHANGELOG . md <nl> ppp b / CHANGELOG . md <nl> <nl> - # # Release 0 . 29 . 1 ( 2019 - 09 - 10 ) <nl> + # # Release 1 . 0 . 0 ( 2019 - 10 - 10 ) <nl> <nl> ` ` ` <nl> - Baseline : 6c5ef5369a3ffceb8a65cc159a2fff1401242810 <nl> + Baseline : 97a82646dadd93bf52d47828bda42e3383b657c6 <nl> <nl> Cherry picks : <nl> <nl> - + 338829f2633e91ae0492ee4169446465e10b5994 : <nl> - Fix retrying of SocketTimeoutExceptions in HttpConnector <nl> - + 14651cd86b6fc1d48f56a208a9b5278b3e2dcf75 : <nl> - Fallback to next urls if download fails in HttpDownloader <nl> - + b7d300c6be3e130dec0e62a4f19493105f595d57 : <nl> - Fix incorrect stdout / stderr in remote action cache . Fixes # 9072 <nl> - + 960217631abdcab0a7ed95e2ab10acd55f636639 : <nl> - Automated rollback of commit <nl> - 0f0a0d58725603cf2f1c175963360b525718a195 . <nl> - + da557f96c697102ad787e57bbf7db2460f6a60a8 : <nl> - Windows : fix " bazel run " argument quoting <nl> - + ef8b6f68cc8ffd2e6523a894034ae383e87ec74c : <nl> - Return JavaInfo from 